In [1]:
import os
os.chdir( "../")

In [3]:
#BEST PARAMS

DICT_RP3 = {'topK': 29, 'alpha': 0.33723025040409343, 'beta': 0.15542352567862933, 'normalize_similarity': True}

DICT_SLIMEN = {'topK': 638, 'l1_ratio': 0.02675165044358113, 'alpha': 0.0014978532379351249} # 0.05018798797659525

DICT_IALS = {'use_gpu': True, 'num_threads' : 1, 'num_factors': 128, 'epochs': 478, 'confidence_scaling': 'linear', 'alpha': 3.358295366834473, 'epsilon': 7.295986722921583, 'reg': 6.292360640597378e-05}
# 0.03486293010333361

DICT_KNN = {'topK': 19, 'shrink': 28, 'similarity': 'tversky', 'normalize': True, 'tversky_alpha': 0.0, 'tversky_beta': 1.3672226785339947}

DICT_HYBIRD = {'alpha1': 0.6464483146524337, 'alpha2': 0.7533801103113421, 'alpha3': 0.032576355148477586, 'min_interaction': 1}

#TRIAL MATRIXES + IALS

In [4]:
import pandas as pd
import numpy as np

path = "Dataset/data_train.csv"
df = pd.read_csv(filepath_or_buffer=path,
                               sep=",",
                               header=1,
                               engine='python',
                               names=['UserID', 'ItemID', 'Interaction'])


df

,UserID,ItemID,Interaction
0,1,15,1.0
1,1,16,1.0
2,1,133,1.0
3,1,161,1.0
4,1,187,1.0
...,...,...,...
478724,13024,13605,1.0
478725,13024,13823,1.0
478726,13024,15122,1.0
478727,13024,18185,1.0


In [5]:
target_users = pd.read_csv("Dataset/data_target_users_test.csv")
target_users.columns = ["UserID"]

In [6]:
user_ids = df["UserID"].unique().tolist()
user2user_encoded = {x: i for i, x in enumerate(user_ids)}
userencoded2user = {i: x for i, x in enumerate(user_ids)}
item_ids = df["ItemID"].unique().tolist()
item2item_encoded = {x: i for i, x in enumerate(item_ids)}
item_encoded2item = {i: x for i, x in enumerate(item_ids)}
df["User"] = df["UserID"].map(user2user_encoded)
df["Item"] = df["ItemID"].map(item2item_encoded)

num_users = len(user2user_encoded)
num_items = len(item_encoded2item)
df["Interaction"] = df["Interaction"].values.astype(np.float32)

# min and max ratings will be used to normalize the ratings later
min_rating = 0.0
max_rating = max(df["Interaction"])

print(
    "Number of users: {}, Number of Items: {}, Min rating: {}, Max rating: {}".format(
        num_users, num_items, min_rating, max_rating
    )
)

Number of users: 12638, Number of Items: 22222, Min rating: 0.0, Max rating: 1.0


In [7]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, KFold
import numpy as np
import scipy.sparse as sps
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample

urm_all = sps.coo_matrix((df["Interaction"].values, 
                          (df["User"].values, df["Item"].values)))

In [8]:
userId_unique = df["UserID"].unique()
itemId_unique = df["ItemID"].unique()

In [9]:
num_users = len(userId_unique)
num_items = len(itemId_unique)

In [10]:
from Recommenders.Recommender_import_list import *
from Recommenders.KNN.ItemKNNSimilarityHybridRecommender import ItemKNNSimilarityHybridRecommender
from Recommenders.Hybrid.GeneralizedLinearHybridRecommenderCold import GeneralizedLinearHybridRecommenderCold
from Recommenders.MatrixFactorization.ImplicitIALSRecommender import ImplicitALSRecommender

2024-01-09 23:33:41.229654: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-01-09 23:33:41.229686: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [11]:
RP3 = RP3betaRecommender(urm_all,verbose=True) 
#RP3.load_model("result_experiments_more_data/Hybrids", RP3.RECOMMENDER_NAME + "_all.zip")
RP3.fit(**DICT_RP3)

SLIMEN = SLIMElasticNetRecommender(urm_all,verbose=True)
#SLIMEN.load_model("result_experiments_more_data/Hybrids", SLIMEN.RECOMMENDER_NAME + "_all.zip")
SLIMEN.fit(**DICT_SLIMEN)

#IALS = ImplicitALSRecommender(urm_all, verbose=True)
#IALS.fit(**{'use_gpu': True, 'num_threads' : 1, 'num_factors': 128, 'epochs': 478, 'confidence_scaling': 'linear', 'alpha': 3.358295366834473, 'epsilon': 7.295986722921583, 'reg': 6.292360640597378e-05})

#KNN = ItemKNNCFRecommender(urm_all, verbose=True)
#KNN.fit(**{'topK': 19, 'shrink': 28, 'similarity': 'tversky', 'normalize': True, 'tversky_alpha': 0.0, 'tversky_beta': 1.3672226785339947})

W1 = RP3.W_sparse
#W1 = (W1 - W1.min()) / (W1.max() - W1.min())

W2 = SLIMEN.W_sparse
#W2 = (W2 - W2.min()) / (W2.max() - W2.min())


#final = ItemKNNSimilarityHybridRecommender(urm_all, W1, W2, verbose=True)
#final.fit(**{'topK': 69, 'alpha': 0.521})

#final = GeneralizedLinearHybridRecommenderCold(urm_all, KNN, [RP3, SLIMEN, IALS],True,1)
#final.fit([0.6464483146524337, 0.7533801103113421, 0.032576355148477586])

RP3betaRecommender: Similarity column 22222 (100.0%), 4536.92 column/sec. Elapsed time 4.90 sec


In [11]:
from Recommenders.Hybrid.GeneralizedLinearCoupleHybridRecommender import GeneralizedLinearCoupleHybridRecommender

alpha = 0.9772360115456751
#minint = hyperparams["min_interaction"]

rec1 = ItemKNNSimilarityHybridRecommender(urm_all, RP3.W_sparse, SLIMEN.W_sparse)
rec1.fit(**{'topK': 1000, 'alpha': 0.446039762079027})

recsys_s = [rec1,IALS]

final = GeneralizedLinearCoupleHybridRecommender(urm_all, recommenders=recsys_s, verbose=True )#COLD_recommender=KNNs[fold])#, COLD_Under_Interactions=minint)
final.fit(alpha)


In [12]:
item_popularity_encoded = np.ediff1d(urm_all.tocsc().indptr)
item_popularity_encoded = np.sort(item_popularity_encoded)

tar_users = target_users["UserID"].astype(int)
topPop_encoded = item_popularity_encoded[-10:]

submission = []

print(np.unique(df["UserID"].values))

for index, user in enumerate(tar_users):
    if (user not in df["UserID"].values):
        item_list_encoded = topPop_encoded
    else:
        item_list_encoded = final.recommend(user2user_encoded[user])[:10]
    item_list = []
    for item_encoded in item_list_encoded:
        item_list.append(item_encoded2item[item_encoded])
    submission.append((user, item_list))


def write_submission(submissions):
    with open("./submission_KFOLDED_mix.csv", "w") as f:
        f.write("user_id,item_list\n")
        for user_id, items in submissions:
            f.write(f"{user_id},{' '.join([str(item) for item in items])}\n")
            
write_submission(submission)

[    1     2     3 ... 13022 13023 13024]
